In [1]:
# Notebook to have all the utility functions defined at one place
# Date written: 18-05-2022
# Written by: Subhamoy Bhaduri

In [2]:
# Import the libraries
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import date
from datetime import timedelta
import sys
import string
from sklearn import datasets, linear_model
pd.set_option('max_colwidth', 800)

In [3]:
# Data pre-processing for primary sales
class dataPreprocessingPrimarySales:
    def typeCast(df):
        """
        Typecast features
        """
    
        df["ForecastUnitCode"] = df["ForecastUnitCode"].astype("str")
        df["EccCustomerLevel7Code"] = df["EccCustomerLevel7Code"].astype("str")
        df["PlantCode"] = df["PlantCode"].astype("str")
        df["Model_Hierarchy"] = df["Model_Hierarchy"].astype("str")
        df["ActualDeliveryDate"] = pd.to_datetime(df["ActualDeliveryDate"])
        df["ActualDeliveredQuantity"] = df["ActualDeliveredQuantity"].astype("float")
    
        return df
    
    def returnAllDates(dt_min, dt_max):
        """
        Return all dates between a date range
        """
        listAllDates = np.array([np.datetime64(dt_min + timedelta(days=x)) for x in range((dt_max - dt_min).days)])
        return listAllDates
  
    def missingDayFill(df, startDate, endDate):
        """
        Insert dummy data with Zero primary sales for all days which are not present in the duration
        """
        fucode = df["ForecastUnitCode"].unique()[0]
        eccl7 = df["EccCustomerLevel7Code"].unique()[0]
        hierarchy = df["Model_Hierarchy"].unique()[0]
        plant = df["PlantCode"].unique()[0]
            
        df = df.sort_values(by="ActualDeliveryDate", ascending=True).reset_index(drop=True)
        #startDate = df["ActualDeliveryDate"].min()
        #endDate = df["ActualDeliveryDate"].max()
        endDate_shifted_by1 = np.datetime64(endDate + timedelta(days=1))
        allDays = dataPreprocessingPrimarySales.returnAllDates(startDate, endDate_shifted_by1)
        if len(allDays) == 0:
            allDays = df["ActualDeliveryDate"].unique()
        
        diff_dates = np.setdiff1d(allDays, df["ActualDeliveryDate"].unique())
                            
        if(len(diff_dates) > 0):
            dict_dummy = {"ForecastUnitCode": np.repeat(fucode, len(diff_dates)),
                          "EccCustomerLevel7Code": np.repeat(eccl7, len(diff_dates)),
                          "PlantCode": np.repeat(plant, len(diff_dates)),
                          "Model_Hierarchy": np.repeat(hierarchy, len(diff_dates)),
                          "ActualDeliveryDate": diff_dates,
                          "ActualDeliveredQuantity": np.repeat(0.0, len(diff_dates))}

            df_dummy = pd.DataFrame(dict_dummy)
            df_new = pd.DataFrame(pd.concat([df, df_dummy], ignore_index=True))
            df_new = df_new.sort_values(by="ActualDeliveryDate", ascending=True).reset_index(drop=True)
        else:
            df_new = df.sort_values(by="ActualDeliveryDate", ascending=True).reset_index(drop=True)
              
        return df_new

    def initialZeroRemoval(df):
        """
        Remove initial zero primary sales so that the time starts from first non zero value
        """
    
        df = df.sort_values(by="ActualDeliveryDate", ascending=True).reset_index(drop=True)
        df["cumsum"] = df["ActualDeliveredQuantity"].cumsum()
        df = df[df["cumsum"] > 0]
        df = df.drop(["cumsum"], axis=1)
    
        return df
  
    def getMOQAndLotSize(df):
        """
        Calculate minimum order quantity and lot sizes using last 28 days of non zero sales from current date
        """
        
        df["MOQ"] = df["ActualDeliveredQuantity"].shift().rolling(28).min()
        df["Lot_Size"] = df["ActualDeliveredQuantity"].shift().rolling(28).mean()
           
        return df

In [4]:
# Enrichment with external data
class enrichmentWithExternalData:
    def getDCDemographicInfo(df, dc_demo_data):
        """
        Enrich primary sales with DC's demographic information
        """
        dc_demo_data["Plnt"] = dc_demo_data["Plnt"].astype("str")
        df = df.merge(dc_demo_data, how="left", left_on="PlantCode", right_on="Plnt")
        df = df.drop(["Plnt", "Place"], axis=1)
    
        return df
    
    def getCovidStringency(df, covid_stringency):
        """
        Enrich primary sales with Covid stringency data
        """
        df["ActualDeliveryDate"] = pd.to_datetime(df["ActualDeliveryDate"])
        covid_stringency["Formatted_Date"] = covid_stringency["Formatted_Date"].astype("str")
        covid_stringency["Formatted_Date"] = pd.to_datetime(covid_stringency["Formatted_Date"])
        df = df.merge(covid_stringency, how="left", left_on=["Full_State_Name", "ActualDeliveryDate"], 
                      right_on=["RegionName", "Formatted_Date"])
        df = df.drop(["CountryName", "RegionName", "Formatted_Date"], axis=1)

        return df
  
    def getGoogleMobility(df, google_covid_mobility):
        """
        Read Covid mobility data and create 3 separate files at County, State and Country levels
        Join first to pull County level data; if not available then use State level data and if still not available then use Country level data
        """
        df["ActualDeliveryDate"] = pd.to_datetime(df["ActualDeliveryDate"])
        google_covid_mobility["date"] = google_covid_mobility["date"].astype("str")
        google_covid_mobility["date"] = pd.to_datetime(google_covid_mobility["date"])
        google_covid_mobility_county = google_covid_mobility[google_covid_mobility["sub_region_2"] != "nan"]
        google_covid_mobility_county = google_covid_mobility_county.drop(["country_region"], axis=1)
        google_covid_mobility_state = google_covid_mobility[(google_covid_mobility["sub_region_2"] == "nan") & (google_covid_mobility["sub_region_1"] != "nan")]
        google_covid_mobility_state = google_covid_mobility_state.drop(["country_region"], axis=1)
        google_covid_mobility_country = google_covid_mobility[(google_covid_mobility["sub_region_2"] == "nan") & (google_covid_mobility["sub_region_1"] == "nan") & (google_covid_mobility["country_region"] != "nan")]
        google_covid_mobility_country = google_covid_mobility_country.drop(["country_region"], axis=1)
           
        df = df.merge(google_covid_mobility_county, how="left", left_on=["Ctr", "Full_State_Name", "County", "ActualDeliveryDate"], 
                      right_on=["country_region_code", "sub_region_1", "sub_region_2", "date"])
        df = df.drop(["country_region_code", "sub_region_1", "sub_region_2", "date"], axis=1)
        df = df.rename(columns={"retail_and_recreation_percent_change_from_baseline": "County_retail_and_recreation_percent_change_from_baseline", 
                                "grocery_and_pharmacy_percent_change_from_baseline": "County_grocery_and_pharmacy_percent_change_from_baseline"})
    
        df = df.merge(google_covid_mobility_state, how="left", left_on=["Ctr", "Full_State_Name", "ActualDeliveryDate"], 
                      right_on=["country_region_code", "sub_region_1", "date"])
        df = df.drop(["country_region_code", "sub_region_1", "sub_region_2", "date"], axis=1)
        df = df.rename(columns={"retail_and_recreation_percent_change_from_baseline": "State_retail_and_recreation_percent_change_from_baseline", 
                                "grocery_and_pharmacy_percent_change_from_baseline": "State_grocery_and_pharmacy_percent_change_from_baseline"})
    
        df = df.merge(google_covid_mobility_country, how="left", left_on=["Ctr", "ActualDeliveryDate"], 
                      right_on=["country_region_code", "date"])
        df = df.drop(["country_region_code", "sub_region_1", "sub_region_2", "date"], axis=1)
        df = df.rename(columns={"retail_and_recreation_percent_change_from_baseline": "Country_retail_and_recreation_percent_change_from_baseline", 
                                "grocery_and_pharmacy_percent_change_from_baseline": "Country_grocery_and_pharmacy_percent_change_from_baseline"})
    
        df["County_retail_and_recreation_percent_change_from_baseline"] = np.where(((df["County_retail_and_recreation_percent_change_from_baseline"] == "nan") | (df["County_retail_and_recreation_percent_change_from_baseline"]).isnull()), df["State_retail_and_recreation_percent_change_from_baseline"], df["County_retail_and_recreation_percent_change_from_baseline"])
    
        df["County_retail_and_recreation_percent_change_from_baseline"] = np.where(((df["County_retail_and_recreation_percent_change_from_baseline"] == "nan") | (df["County_retail_and_recreation_percent_change_from_baseline"]).isnull()), df["Country_retail_and_recreation_percent_change_from_baseline"], df["County_retail_and_recreation_percent_change_from_baseline"])
    
        df["County_grocery_and_pharmacy_percent_change_from_baseline"] = np.where(((df["County_grocery_and_pharmacy_percent_change_from_baseline"] == "nan") | (df["County_grocery_and_pharmacy_percent_change_from_baseline"]).isnull()), df["State_grocery_and_pharmacy_percent_change_from_baseline"], df["County_grocery_and_pharmacy_percent_change_from_baseline"])
    
        df["County_grocery_and_pharmacy_percent_change_from_baseline"] = np.where(((df["County_grocery_and_pharmacy_percent_change_from_baseline"] == "nan") | (df["County_grocery_and_pharmacy_percent_change_from_baseline"]).isnull()), df["Country_grocery_and_pharmacy_percent_change_from_baseline"], df["County_grocery_and_pharmacy_percent_change_from_baseline"])
    
        df = df.rename(columns={"County_retail_and_recreation_percent_change_from_baseline": "retail_and_recreation_percent_change_from_baseline", 
                                "County_grocery_and_pharmacy_percent_change_from_baseline": "grocery_and_pharmacy_percent_change_from_baseline"})

        df = df.drop(["State_retail_and_recreation_percent_change_from_baseline", "State_grocery_and_pharmacy_percent_change_from_baseline",
                      "Country_retail_and_recreation_percent_change_from_baseline", "Country_grocery_and_pharmacy_percent_change_from_baseline"], 
                     axis=1)
    
        return df
  
    def getHolidayInfo(df, holiday_info):
        """
        Enrich primary sales with holiday information
        Create 2 separate holiday dataframe at federal and state levels
        """
        df["ActualDeliveryDate"] = pd.to_datetime(df["ActualDeliveryDate"])
        holiday_info["Date"] = holiday_info["Date"].astype("str")
        holiday_info["Date"] = pd.to_datetime(holiday_info["Date"])
        # Create 2 separate holiday dataframe at federal and state levels
        holiday_info_fed_pre = holiday_info[holiday_info["Jurisdiction"] == "Federal"]
        holiday_info_fed = holiday_info_fed_pre.copy()
        holiday_info_fed["isFedHoliday"] = 1
        holiday_info_fed = holiday_info_fed.rename(columns={"Holiday Description": "Fed_Holiday_Desc"})
        
        """
        holiday_info_state_pre = holiday_info[holiday_info["Jurisdiction"] != "Federal"]
        holiday_info_state = holiday_info_state_pre.copy()
        holiday_info_state["isStateHoliday"] = 1
        holiday_info_state = holiday_info_state.rename(columns={"Holiday Description": "State_Holiday_Desc"})
        
        df = df.merge(holiday_info_state, how="left", left_on=["ActualDeliveryDate", "Ctr", "Rg"], 
                      right_on=["Date", "Country", "Jurisdiction"])
        df = df.drop(["Date", "Jurisdiction", "Country"], axis=1)
        """
    
        df = df.merge(holiday_info_fed, how="left", left_on=["ActualDeliveryDate", "Ctr"], 
                      right_on=["Date", "Country"])
        df = df.drop(["Date", "Jurisdiction", "Country"], axis=1)
    
        #df["isHoliday"] = np.where(((df["isFedHoliday"] == 1) | (df["isStateHoliday"] == 1)), "Yes", "No")
        df["isHoliday"] = np.where(((df["isFedHoliday"] == 1)), "Yes", "No")
        #df["Holiday_Desc"] = np.where(df["Fed_Holiday_Desc"].notnull(), df['Fed_Holiday_Desc'], df["State_Holiday_Desc"])
        df["Holiday_Desc"] = df["Fed_Holiday_Desc"]

        #df = df.drop(["isFedHoliday", "isStateHoliday", "Fed_Holiday_Desc", "State_Holiday_Desc"], axis=1)
        df = df.drop(["isFedHoliday", "Fed_Holiday_Desc"], axis=1)
    
        return df
  
    def getWeatherInfo(df):
        """
        Enrich primary sales with weather information
        """
    
        springList = [3, 4, 5]
        summerList = [6, 7, 8]
        fallList = [9, 10, 11]
        winterList = [12, 1, 2]
    
        df["Month"] = pd.to_datetime(df["ActualDeliveryDate"]).dt.month
        df["Month"] = df["Month"].astype("int")
        df["Season"] = df["Month"].apply(lambda x: "Spring" if x in springList else ("Summer" if x in summerList else ("Fall" if x in fallList else "Winter")))
        df = df.drop(["Month"], axis=1)
    
        return df

In [5]:
# Further data processing
class dataAdditionalprocessing:
    def typeCast(df):
        """
        Typecast features
        """
    
        df["Model_Hierarchy"] = df["Model_Hierarchy"].astype("str")
        df["ForecastUnitCode"] = df["ForecastUnitCode"].astype("str")
        df["EccCustomerLevel7Code"] = df["EccCustomerLevel7Code"].astype("str")
        df["PlantCode"] = df["PlantCode"].astype("str")
        df["ActualDeliveryDate"] = pd.to_datetime(df["ActualDeliveryDate"])
        df["ActualDeliveredQuantity"] = df["ActualDeliveredQuantity"].astype("float")
        df["StringencyIndex"] = df["StringencyIndex"].astype("float")
        df["retail_and_recreation_percent_change_from_baseline"] = df["retail_and_recreation_percent_change_from_baseline"].astype("float")
        df["grocery_and_pharmacy_percent_change_from_baseline"] = df["grocery_and_pharmacy_percent_change_from_baseline"].astype("float")
        df["isHoliday"] = df["isHoliday"].astype("str")
        df["Holiday_Desc"] = df["Holiday_Desc"].astype("str")
        df["Season"] = df["Season"].astype("str")
        df["MOQ"] = df["MOQ"].astype("float")
        df["Lot_Size"] = df["Lot_Size"].astype("float")
        df["Month"] = df["Month"].astype("int")
        df["Days_Of_Week"] = df["Days_Of_Week"].astype("int")
        df["isWeekend"] = df["isWeekend"].astype("str")
        df["Day"] = df["Day"].astype("int")
        df["isMonthStartDate"] = df["isMonthStartDate"].astype("str")
        df["isMonthEndDate"] = df["isMonthEndDate"].astype("str")
        df["isQtrStartDate"] = df["isQtrStartDate"].astype("str")
        df["isQtrEndDate"] = df["isQtrEndDate"].astype("str")
        df["isBeforeHoliday"] = df["isBeforeHoliday"].astype("str")
        df["isAfterHoliday"] = df["isAfterHoliday"].astype("str")
    
        return df
  
    def fillMissingValue(df):
        """
        Impute missing values
        """
    
        df["ActualDeliveredQuantity"] = df["ActualDeliveredQuantity"].fillna(0.0)
        df["StringencyIndex"] = df["StringencyIndex"].fillna(0.0)
        df["StringencyIndex"] = df["StringencyIndex"].replace(np.nan, 0.0)
        df["MOQ"] = df["MOQ"].fillna(0.0)
        df["Lot_Size"] = df["Lot_Size"].fillna(0.0)
        df["retail_and_recreation_percent_change_from_baseline"] = df["retail_and_recreation_percent_change_from_baseline"].fillna(0.0)
        df["grocery_and_pharmacy_percent_change_from_baseline"] = df["grocery_and_pharmacy_percent_change_from_baseline"].fillna(0.0)
        df["Holiday_Desc"] = df["Holiday_Desc"].replace(to_replace=["None"], value=np.nan)
        df["Holiday_Desc"] = df["Holiday_Desc"].fillna("No_Holiday")
        
        return df
  
    def formatHolidayDesc(val):
        """
        Remove unwanted characters and replace spaces with underscores
        """
    
        new_val = val.replace("'","").replace(".", "").replace("(", "").replace(")", "").replace(" ", "_")
    
        return str(new_val)
  
    def identifyHolidayPeriod(df, holiday_info):
        """
        Mark 7 days before/after of any holiday as a holiday period
        """
        
        holiday_info["Date"] = holiday_info["Date"].astype("str")
        holiday_info["Date"] = pd.to_datetime(holiday_info["Date"])
        holiday_info_fed = holiday_info[holiday_info["Jurisdiction"] == "Federal"]
        holiday_info_US = holiday_info_fed.copy()
        holiday_info_CA = holiday_info_fed.copy()
        holiday_info_US = holiday_info_US[holiday_info_US["Country"] == "US"]
        holiday_info_CA = holiday_info_CA[holiday_info_CA["Country"] == "CA"]
        df["ActualDeliveryDate"] = pd.to_datetime(df["ActualDeliveryDate"])
        df_US = df.copy()
        df_CA = df.copy()
        df_US = df_US[df_US["Ctr"] == "US"]
        df_CA = df_CA[df_CA["Ctr"] == "CA"]
                
        if df_US.shape[0] > 0:
            df_US["isAfterHoliday"] = "No"
            df_US["isBeforeHoliday"] = "No"
            df_US["isAfterHoliday"] = np.where((df_US["ActualDeliveryDate"].isin(holiday_info_US["Date"] + pd.DateOffset(days=1))), "Yes", df_US["isAfterHoliday"])
            df_US["isAfterHoliday"] = np.where((df_US["ActualDeliveryDate"].isin(holiday_info_US["Date"] + pd.DateOffset(days=2))), "Yes", df_US["isAfterHoliday"])
            df_US["isAfterHoliday"] = np.where((df_US["ActualDeliveryDate"].isin(holiday_info_US["Date"] + pd.DateOffset(days=3))), "Yes", df_US["isAfterHoliday"])
            df_US["isAfterHoliday"] = np.where((df_US["ActualDeliveryDate"].isin(holiday_info_US["Date"] + pd.DateOffset(days=4))), "Yes", df_US["isAfterHoliday"])
            df_US["isAfterHoliday"] = np.where((df_US["ActualDeliveryDate"].isin(holiday_info_US["Date"] + pd.DateOffset(days=5))), "Yes", df_US["isAfterHoliday"])
            df_US["isAfterHoliday"] = np.where((df_US["ActualDeliveryDate"].isin(holiday_info_US["Date"] + pd.DateOffset(days=6))), "Yes", df_US["isAfterHoliday"])
            df_US["isAfterHoliday"] = np.where((df_US["ActualDeliveryDate"].isin(holiday_info_US["Date"] + pd.DateOffset(days=7))), "Yes", df_US["isAfterHoliday"])
            df_US["isBeforeHoliday"] = np.where((df_US["ActualDeliveryDate"].isin(holiday_info_US["Date"] - pd.DateOffset(days=1))), "Yes", df_US["isBeforeHoliday"])
            df_US["isBeforeHoliday"] = np.where((df_US["ActualDeliveryDate"].isin(holiday_info_US["Date"] - pd.DateOffset(days=2))), "Yes", df_US["isBeforeHoliday"])
            df_US["isBeforeHoliday"] = np.where((df_US["ActualDeliveryDate"].isin(holiday_info_US["Date"] - pd.DateOffset(days=3))), "Yes", df_US["isBeforeHoliday"])
            df_US["isBeforeHoliday"] = np.where((df_US["ActualDeliveryDate"].isin(holiday_info_US["Date"] - pd.DateOffset(days=4))), "Yes", df_US["isBeforeHoliday"])
            df_US["isBeforeHoliday"] = np.where((df_US["ActualDeliveryDate"].isin(holiday_info_US["Date"] - pd.DateOffset(days=5))), "Yes", df_US["isBeforeHoliday"])
            df_US["isBeforeHoliday"] = np.where((df_US["ActualDeliveryDate"].isin(holiday_info_US["Date"] - pd.DateOffset(days=6))), "Yes", df_US["isBeforeHoliday"])
            df_US["isBeforeHoliday"] = np.where((df_US["ActualDeliveryDate"].isin(holiday_info_US["Date"] - pd.DateOffset(days=7))), "Yes", df_US["isBeforeHoliday"])
                       
        if df_CA.shape[0] > 0:
            df_CA["isAfterHoliday"] = "No"
            df_CA["isBeforeHoliday"] = "No"
            df_CA["isAfterHoliday"] = np.where((df_CA["ActualDeliveryDate"].isin(holiday_info_CA["Date"] + pd.DateOffset(days=1))), "Yes", df_CA["isAfterHoliday"])
            df_CA["isAfterHoliday"] = np.where((df_CA["ActualDeliveryDate"].isin(holiday_info_CA["Date"] + pd.DateOffset(days=2))), "Yes", df_CA["isAfterHoliday"])
            df_CA["isAfterHoliday"] = np.where((df_CA["ActualDeliveryDate"].isin(holiday_info_CA["Date"] + pd.DateOffset(days=3))), "Yes", df_CA["isAfterHoliday"])
            df_CA["isAfterHoliday"] = np.where((df_CA["ActualDeliveryDate"].isin(holiday_info_CA["Date"] + pd.DateOffset(days=4))), "Yes", df_CA["isAfterHoliday"])
            df_CA["isAfterHoliday"] = np.where((df_CA["ActualDeliveryDate"].isin(holiday_info_CA["Date"] + pd.DateOffset(days=5))), "Yes", df_CA["isAfterHoliday"])
            df_CA["isAfterHoliday"] = np.where((df_CA["ActualDeliveryDate"].isin(holiday_info_CA["Date"] + pd.DateOffset(days=6))), "Yes", df_CA["isAfterHoliday"])
            df_CA["isAfterHoliday"] = np.where((df_CA["ActualDeliveryDate"].isin(holiday_info_CA["Date"] + pd.DateOffset(days=7))), "Yes", df_CA["isAfterHoliday"])
            df_CA["isBeforeHoliday"] = np.where((df_CA["ActualDeliveryDate"].isin(holiday_info_CA["Date"] - pd.DateOffset(days=1))), "Yes", df_CA["isBeforeHoliday"])
            df_CA["isBeforeHoliday"] = np.where((df_CA["ActualDeliveryDate"].isin(holiday_info_CA["Date"] - pd.DateOffset(days=2))), "Yes", df_CA["isBeforeHoliday"])
            df_CA["isBeforeHoliday"] = np.where((df_CA["ActualDeliveryDate"].isin(holiday_info_CA["Date"] - pd.DateOffset(days=3))), "Yes", df_CA["isBeforeHoliday"])
            df_CA["isBeforeHoliday"] = np.where((df_CA["ActualDeliveryDate"].isin(holiday_info_CA["Date"] - pd.DateOffset(days=4))), "Yes", df_CA["isBeforeHoliday"])
            df_CA["isBeforeHoliday"] = np.where((df_CA["ActualDeliveryDate"].isin(holiday_info_CA["Date"] - pd.DateOffset(days=5))), "Yes", df_CA["isBeforeHoliday"])
            df_CA["isBeforeHoliday"] = np.where((df_CA["ActualDeliveryDate"].isin(holiday_info_CA["Date"] - pd.DateOffset(days=6))), "Yes", df_CA["isBeforeHoliday"])
            df_CA["isBeforeHoliday"] = np.where((df_CA["ActualDeliveryDate"].isin(holiday_info_CA["Date"] - pd.DateOffset(days=7))), "Yes", df_CA["isBeforeHoliday"])   
               
        if ((df_US.shape[0] > 0) & (df_CA.shape[0] > 0)):
            return_df = pd.concat([df_US, df_CA], axis=0)
        elif (df_US.shape[0] > 0):
            return_df = df_US          
        else:
            return_df = df_CA
                                           
        return return_df
  
    def featureEnggDate(df):
        """
        Add custom features from Date
        """
    
        df["ActualDeliveryDate"] = pd.to_datetime(df["ActualDeliveryDate"])
        df["Month"] = pd.to_datetime(df["ActualDeliveryDate"]).dt.month
        df["Days_Of_Week"] = pd.to_datetime(df["ActualDeliveryDate"]).dt.dayofweek
        df["isWeekend"] = df["Days_Of_Week"].apply(lambda x: "Yes" if ((x == 5) | (x == 6)) else "No")
        df["Day"] = pd.to_datetime(df["ActualDeliveryDate"]).dt.day
        
        startYear = df["ActualDeliveryDate"].min().year
        endYear = df["ActualDeliveryDate"].max().year
        startPeriod = date(startYear, 1, 1)
        endPeriod = date(endYear, 12, 31)
        monthStartDate = pd.date_range(startPeriod, endPeriod, freq="MS")
        monthEndDate = pd.date_range(startPeriod, endPeriod, freq="M")
        qtrStartDate = pd.date_range(startPeriod, endPeriod, freq="QS")
        qtrEndDate = pd.date_range(startPeriod, endPeriod, freq="Q") 
    
        dfMonth = pd.DataFrame(list(zip(monthStartDate, monthEndDate)), columns=["Month_Start_Date", "Month_End_Date"])
        dfQtr = pd.DataFrame(list(zip(qtrStartDate, qtrEndDate)), columns=["Qtr_Start_Date", "Qtr_End_Date"])
    
        dfMonth["Month_Start_Date"] = pd.to_datetime(dfMonth["Month_Start_Date"]).dt.date
        dfMonth["Month_End_Date"] = pd.to_datetime(dfMonth["Month_End_Date"]).dt.date
        dfQtr["Qtr_Start_Date"] = pd.to_datetime(dfQtr["Qtr_Start_Date"]).dt.date
        dfQtr["Qtr_End_Date"] = pd.to_datetime(dfQtr["Qtr_End_Date"]).dt.date
    
        df["isMonthStartDate"] = np.where(df["ActualDeliveryDate"].isin(monthStartDate), "Yes", "No")
        df["isMonthEndDate"] = np.where(df["ActualDeliveryDate"].isin(monthEndDate), "Yes", "No")
        df["isQtrStartDate"] = np.where(df["ActualDeliveryDate"].isin(qtrStartDate), "Yes", "No")
        df["isQtrEndDate"] = np.where(df["ActualDeliveryDate"].isin(qtrEndDate), "Yes", "No")   
               
        df["ActualDeliveryDate"] = pd.to_datetime(df["ActualDeliveryDate"]).dt.date
    
        return df  

    def featureEnggLag(df):
        """
        Add Lag features
        """
        
        df["Lag_1"] = df["ActualDeliveredQuantity"].shift(periods=7, fill_value=0.0)
        df["Lag_2"] = df["ActualDeliveredQuantity"].shift(periods=14, fill_value=0.0)
        df["Lag_3"] = df["ActualDeliveredQuantity"].shift(periods=21, fill_value=0.0)
        df["Lag_4"] = df["ActualDeliveredQuantity"].shift(periods=28, fill_value=0.0)
        df["Lag_5"] = df["ActualDeliveredQuantity"].shift(periods=35, fill_value=0.0)
        df["Lag_6"] = df["ActualDeliveredQuantity"].shift(periods=42, fill_value=0.0)
        df["Lag_7"] = df["ActualDeliveredQuantity"].shift(periods=49, fill_value=0.0)
        df["Lag_8"] = df["ActualDeliveredQuantity"].shift(periods=56, fill_value=0.0)
        df["Lag_9"] = df["ActualDeliveredQuantity"].shift(periods=63, fill_value=0.0)
        df["Lag_10"] = df["ActualDeliveredQuantity"].shift(periods=70, fill_value=0.0)
        df["Lag_11"] = df["ActualDeliveredQuantity"].shift(periods=77, fill_value=0.0)
        df["Lag_12"] = df["ActualDeliveredQuantity"].shift(periods=84, fill_value=0.0)
        df["Lag_13"] = df["ActualDeliveredQuantity"].shift(periods=91, fill_value=0.0)
    
        """
        X = df[['Day']]
        y = df[['ActualDeliveredQuantity']]
        regr = linear_model.LinearRegression()
        fit = regr.fit(X, y)    
        predicted_volume = pd.DataFrame(regr.predict(X))
        predicted_volume = pd.DataFrame(predicted_volume.rename(columns={0: "Predicted_Volume"}))
        df = pd.concat([df.reset_index(drop=True), predicted_volume], axis=1)  
        df["Predicted_Volume"] = np.where(df["Predicted_Volume"] < 0, 0, df["Predicted_Volume"])
        df["Trend_Factor"] = df["Predicted_Volume"]/df["ActualDeliveredQuantity"][0]
        df["Detrended_Volume"] = df["ActualDeliveredQuantity"]/df["Trend_Factor"]
        df["Detrended_Volume"] = df["Detrended_Volume"].fillna(0.0)
        df = df.replace(np.inf, 0.0)
        df["91days_MA"] = df["Detrended_Volume"].rolling(window=28).mean()
        df["91days_SI"] = (df["91days_MA"]/df["91days_MA"].mean())-1
        df = df.drop(["Predicted_Volume", "Detrended_Volume", "91days_MA"], axis=1)
        df["Trend_Factor"] = df["Trend_Factor"].astype("float")
        df["91days_SI"] = df["91days_SI"].astype("float")
        df["Trend_Factor"] = df["Trend_Factor"].fillna(0.0)
        df["91days_SI"] = df["91days_SI"].fillna(0.0)
        """
        
        return df
            
    def primarySalesCapping(df):
        """
        Outlier treatment for primary sales
        """

        #df["ActualDeliveredQuantity"] = df["ActualDeliveredQuantity"].apply(lambda x: x if x < percentile_90 else percentile_90)
        percentile_90 = np.percentile(df["ActualDeliveredQuantity"], 90)
        if percentile_90 != 0.0:
            df["Outlier_Ind"] = np.where(df["ActualDeliveredQuantity"] > percentile_90, "True", "False")
            df["ActualDeliveredQuantity"] = np.where(df["Outlier_Ind"] == "True", percentile_90, df["ActualDeliveredQuantity"])
            df = df.drop(["Outlier_Ind"], axis=1)
    
        return df

In [6]:
# Data generation for future forecasting
class futureDataGeneration:
    def genFutureDataStepA(df, endDate, dc_demo_data, holiday_info):
        """
        Generate data for future forecast
        """
        
        df_train = df.copy()
        start_date = np.datetime64(endDate + timedelta(days=2))
        date_range = pd.date_range(start=start_date, periods=28).to_pydatetime().tolist()
        future_df = pd.DataFrame(date_range, columns=["ActualDeliveryDate"])
        future_df["ActualDeliveryDate"] = pd.to_datetime(future_df["ActualDeliveryDate"]).dt.date
        future_df["ForecastUnitCode"] = None
        future_df["EccCustomerLevel7Code"] = None
        future_df["Model_Hierarchy"] = None
        future_df["ActualDeliveredQuantity"] = 0.0
        future_df["PlantCode"] = None
        future_df["MOQ"] = None
        future_df["Lot_Size"] = None
        future_df["StringencyIndex"] = None
        future_df["retail_and_recreation_percent_change_from_baseline"] = None
        future_df["grocery_and_pharmacy_percent_change_from_baseline"] = None
        
        df = df[['ForecastUnitCode', 'EccCustomerLevel7Code', 'Model_Hierarchy', 
                 'ActualDeliveryDate', 'ActualDeliveredQuantity', 'PlantCode', 'MOQ', 
                 'Lot_Size', 'StringencyIndex', 
                 'retail_and_recreation_percent_change_from_baseline', 
                 'grocery_and_pharmacy_percent_change_from_baseline']]
        future_df = future_df[['ForecastUnitCode', 'EccCustomerLevel7Code', 'Model_Hierarchy', 
                               'ActualDeliveryDate', 'ActualDeliveredQuantity', 'PlantCode', 'MOQ', 
                               'Lot_Size', 'StringencyIndex', 
                               'retail_and_recreation_percent_change_from_baseline', 
                               'grocery_and_pharmacy_percent_change_from_baseline']]
        future_df = future_df.astype(df.dtypes.to_dict())
                
        future_df = pd.concat([df, future_df], axis=0)
        future_df = future_df.ffill(axis=0)
        future_df["ActualDeliveryDate"] = pd.to_datetime(future_df["ActualDeliveryDate"]).astype("str")
        future_df["ActualDeliveryDate"] = pd.to_datetime(future_df["ActualDeliveryDate"])
        future_df = future_df[future_df["ActualDeliveryDate"] > endDate]
        
        future_df_withDC = enrichmentWithExternalData.getDCDemographicInfo(future_df, dc_demo_data)
        
        future_df_withHoliday = enrichmentWithExternalData.getHolidayInfo(future_df_withDC, holiday_info)
        
        future_df_withWeather = enrichmentWithExternalData.getWeatherInfo(future_df_withHoliday)
        
        future_df_EnrichedExternal = future_df_withWeather.copy()
        future_df_EnrichedExternal = future_df_EnrichedExternal[["ForecastUnitCode", "EccCustomerLevel7Code", 
                                                                 "Model_Hierarchy", "ActualDeliveryDate", 
                                                                 "ActualDeliveredQuantity", "PlantCode", 
                                                                 "MOQ", "Lot_Size", "StringencyIndex", "Ctr", 
                                                                 "retail_and_recreation_percent_change_from_baseline", 
                                                                 "grocery_and_pharmacy_percent_change_from_baseline", "isHoliday", 
                                                                 "Holiday_Desc", "Season"]]
        
        future_df_EnrichedExternal = dataAdditionalprocessing.fillMissingValue(future_df_EnrichedExternal)
        future_df_EnrichedExternal["Holiday_Desc"] = future_df_EnrichedExternal["Holiday_Desc"].apply(dataAdditionalprocessing.formatHolidayDesc)
        future_df_EnrichedExternal = future_df_EnrichedExternal.sort_values(by=["Model_Hierarchy", "ActualDeliveryDate"], ascending=True).reset_index(drop=True)    
        
        return future_df_EnrichedExternal
        
    def genFutureDataStepB(x):
        """
        Generate data for future forecast
        """
        
        a, b = dataAdditionalprocessing.identifyHolidayPeriod(x)
                               
        return (a, b)
        
    def genFutureDataStepC(df):
        """
        Generate data for future forecast
        """
        
        future_df_featureEngineered = dataAdditionalprocessing.featureEnggDate(df)
        future_df_featureEngineered = dataAdditionalprocessing.featureEnggLag(future_df_featureEngineered)
    
        future_df_featureEngineered.loc[7:, "Lag_1"] = df["Lag_1"].tolist()[-1]
        future_df_featureEngineered.loc[14:, "Lag_2"] = df["Lag_2"].tolist()[-1]
        future_df_featureEngineered.loc[21:, "Lag_3"] = df["Lag_3"].tolist()[-1]
        future_df_featureEngineered.loc[28:, "Lag_4"] = df["Lag_4"].tolist()[-1]
        future_df_featureEngineered["Lag_5"] = 0.0
        future_df_featureEngineered["Lag_6"] = 0.0
        future_df_featureEngineered["Lag_7"] = 0.0
        future_df_featureEngineered["Lag_8"] = 0.0
        future_df_featureEngineered["Lag_9"] = 0.0
        future_df_featureEngineered["Lag_10"] = 0.0
        future_df_featureEngineered["Lag_11"] = 0.0
        future_df_featureEngineered["Lag_12"] = 0.0
        future_df_featureEngineered["Lag_13"] = 0.0
        future_df_typeCastFinal = dataAdditionalprocessing.typeCast(future_df_featureEngineered)
        # Drop additional columns
        future_df_typeCastFinal = future_df_typeCastFinal.drop(["isHoliday", "Ctr"], axis=1)
            
        return future_df_typeCastFinal